In [1]:
import os
import numpy as np
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
from tensorflow.keras import models
import tensorflow as tf
from focal_loss import BinaryFocalLoss

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [3]:
model_dir ='/media/tohn/SSD/ModelTrainByImages/R2_8/models/B5R2b5_AN_1FC_fold8_1.h5'
model = load_model(model_dir)
height = width = model.input_shape[1]

x = model.get_layer('head_pooling').output
prediction_layer = model.output
model2 = models.Model(inputs= model.input, outputs=[prediction_layer,x]) 

In [4]:
#ไม่ต้องเปลี่ยนก็ได้ เพราะเอา Label มาใช้เฉยๆ 
# validation
import pandas as pd
base_dir  = '/media/tohn/SSD/Images/Image1/validation/'
dataframe = pd.read_csv( '/home/yupaporn/codes/USAI/Validationdf_fold1_3.csv')
validation_dir = os.path.join(base_dir, 'validation')

#Train
train_df = pd.read_csv( '/home/yupaporn/codes/USAI/Traindf_fold1_3.csv')
base_dir0 = '/media/tohn/SSD/Images/Image1/'
os.chdir(base_dir0)
train_dir = os.path.join(base_dir0, 'train')

In [5]:
import pandas as pd
df0 = pd.read_csv (r'/home/yupaporn/codes/USAI/Validationdf_fold8.csv')
print(df0 .shape)
dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
# a = df01[df01['Case'].between(1, 10)]
# dataframe = df01[df01['Abs Position']!='P8']
print(dataframe.shape)
print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)
dataframe.head(5)

(656, 26)
(656, 26)
Normal:  (428, 26)
Abnormal:  (228, 26)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Case,Abs Position,Sub Position,Class,Sub_class,...,left,top,width,height,Rleft,Rtop,Rwidth,Rheight,FileName,filename
0,0,0,0,215,215,20,P3,P31,Abnormal,AB01,...,152.0,96.0,617.0,548.0,0.153985,0.107041,0.623188,0.609562,AB01 P3-1 C020.JPG,AB01 P3-1 C020.JPG
1,1,1,1,216,216,20,P2,P2,Abnormal,AB01,...,233.0,119.0,568.0,512.0,0.234657,0.132937,0.572202,0.569721,AB01 P2 C020.JPG,AB01 P2 C020.JPG
2,2,2,2,217,217,20,P5,P51,Abnormal,AB01,...,213.0,135.0,568.0,640.0,0.215190,0.150865,0.573237,0.711155,AB01 P5-1 C020.JPG,AB01 P5-1 C020.JPG
3,3,3,3,218,218,20,P1,P1,Abnormal,AB01,...,186.0,65.0,593.0,541.0,0.185053,0.073176,0.588968,0.601594,AB01 P1 C020.JPG,AB01 P1 C020.JPG
4,4,4,4,219,219,20,P5,P52,Abnormal,AB01,...,80.0,132.0,590.0,491.0,0.081967,0.146881,0.599271,0.545817,AB01 P5-2 C020.JPG,AB01 P5-2 C020.JPG


In [6]:
# dataframe.info()

In [6]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Class',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

#label
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

Found 656 validated image filenames belonging to 2 classes.
{0: 'Abnormal', 1: 'Normal'}


In [7]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model2.predict([x])
    result1 = result[0][0]
    result2 = result[1][0]

    return result1, result2

#Predict
pred_list = list()
prob_list = list()
vectorlist = list()
img_path=dataframe['Path Crop'].tolist()
for i in range(0,len(img_path)):
    result1,result2 = predict_image(img_path[i])
    result = np.argmax(result1)
    pred_list.append(labels[result])
    prob_list.append(result1[result])
    vectorlist.append(result2.tolist())
    
dataframe['category'] = pred_list
dataframe['Prob'] = prob_list

In [8]:
vector_array = np.array(vectorlist)
vector_array

array([[-1.45873189e-01, -1.45606831e-01,  2.25965846e-02, ...,
        -9.62207019e-02,  8.31721276e-02, -1.22259639e-01],
       [-1.60234764e-01, -1.48495942e-01,  1.38017252e-01, ...,
        -1.12319604e-01, -2.33845585e-05, -8.84213969e-02],
       [-1.54414400e-01, -1.26035780e-01,  1.52980536e-01, ...,
        -1.35873422e-01,  1.85502455e-01, -1.37309939e-01],
       ...,
       [ 1.70927241e-01,  2.81547457e-01, -1.86315253e-01, ...,
         5.88554554e-02, -1.29015863e-01,  2.87848592e-01],
       [-3.34926546e-02, -8.70545488e-03, -1.83772564e-01, ...,
         2.34812289e-01, -3.37920710e-02,  4.05666828e-01],
       [ 1.76601678e-01,  7.94264555e-01, -8.45282748e-02, ...,
         6.82115972e-01, -4.95530069e-02,  8.93054962e-01]])

In [9]:
dataset = pd.DataFrame(vector_array[:, :])
dataset

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,-0.145873,-0.145607,0.022597,-0.027057,-0.200997,-0.137499,0.040337,-0.171097,-0.207573,-0.009216,...,-0.129475,-0.096019,-0.002620,-0.155927,-0.168271,-0.040307,-0.008302,-0.096221,0.083172,-0.122260
1,-0.160235,-0.148496,0.138017,-0.097721,-0.192524,-0.158592,0.136529,-0.207999,-0.185892,-0.068355,...,-0.140916,0.024096,0.004154,-0.090007,-0.180010,-0.046805,0.049341,-0.112320,-0.000023,-0.088421
2,-0.154414,-0.126036,0.152981,-0.134474,-0.222772,-0.042759,0.274228,-0.077468,-0.208737,-0.076165,...,-0.116578,0.032991,0.133067,-0.143042,-0.161731,-0.070467,-0.029572,-0.135873,0.185502,-0.137310
3,-0.122253,-0.164240,0.291866,-0.138341,-0.158042,-0.132671,0.424885,-0.184487,-0.209159,0.033796,...,-0.147574,0.070383,0.068945,-0.132652,-0.192535,-0.043425,0.225481,-0.125891,0.087897,0.018063
4,-0.068276,-0.090568,0.176315,-0.132916,-0.101672,-0.127494,0.184186,-0.205508,-0.188595,0.052127,...,-0.082124,0.049073,0.176685,-0.105538,-0.188592,-0.050513,-0.077762,-0.041359,0.148492,-0.057217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,-0.065322,-0.096514,-0.185740,0.018262,-0.105873,0.100438,-0.123069,0.066184,-0.119514,-0.118816,...,0.219946,-0.012008,-0.165648,0.104313,-0.000023,-0.113952,-0.061757,-0.120490,-0.199286,-0.045103
652,0.295571,0.307370,-0.139219,0.676659,0.341264,0.466379,-0.127595,1.030663,0.489739,-0.138380,...,0.539170,-0.140058,-0.146595,0.442800,0.249083,-0.172375,0.089985,0.115840,-0.089799,0.482417
653,0.170927,0.281547,-0.186315,0.508857,0.264922,0.381621,-0.090961,0.442370,0.262251,-0.103593,...,0.356563,-0.172619,-0.173975,0.317737,0.589979,0.003480,-0.108289,0.058855,-0.129016,0.287849
654,-0.033493,-0.008705,-0.183773,0.520157,0.214841,-0.208221,-0.053012,-0.036728,0.262903,-0.113315,...,-0.037537,-0.174654,-0.099289,0.117439,-0.131957,0.284193,-0.023389,0.234812,-0.033792,0.405667


In [10]:
a = dataframe['Class'].tolist()
b = dataframe['Sub Position'].tolist()
c = dataframe['Views'].tolist()
d = dataframe['Sub_class'].tolist()
dataset['Class'] = a 
dataset['SubPosition'] = b
dataset['Views'] = c
dataset['Sub_class'] = d

In [11]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,2042,2043,2044,2045,2046,2047,Class,SubPosition,Views,Sub_class
0,-0.145873,-0.145607,0.022597,-0.027057,-0.200997,-0.137499,0.040337,-0.171097,-0.207573,-0.009216,...,-0.168271,-0.040307,-0.008302,-0.096221,0.083172,-0.122260,Abnormal,P31,FP-B,AB01
1,-0.160235,-0.148496,0.138017,-0.097721,-0.192524,-0.158592,0.136529,-0.207999,-0.185892,-0.068355,...,-0.180010,-0.046805,0.049341,-0.112320,-0.000023,-0.088421,Abnormal,P2,FP-A,AB01
2,-0.154414,-0.126036,0.152981,-0.134474,-0.222772,-0.042759,0.274228,-0.077468,-0.208737,-0.076165,...,-0.161731,-0.070467,-0.029572,-0.135873,0.185502,-0.137310,Abnormal,P51,FP-C,AB01
3,-0.122253,-0.164240,0.291866,-0.138341,-0.158042,-0.132671,0.424885,-0.184487,-0.209159,0.033796,...,-0.192535,-0.043425,0.225481,-0.125891,0.087897,0.018063,Abnormal,P1,FP-A,AB01
4,-0.068276,-0.090568,0.176315,-0.132916,-0.101672,-0.127494,0.184186,-0.205508,-0.188595,0.052127,...,-0.188592,-0.050513,-0.077762,-0.041359,0.148492,-0.057217,Abnormal,P52,FP-C,AB01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,-0.065322,-0.096514,-0.185740,0.018262,-0.105873,0.100438,-0.123069,0.066184,-0.119514,-0.118816,...,-0.000023,-0.113952,-0.061757,-0.120490,-0.199286,-0.045103,Normal,P71,FP-D,Normal
652,0.295571,0.307370,-0.139219,0.676659,0.341264,0.466379,-0.127595,1.030663,0.489739,-0.138380,...,0.249083,-0.172375,0.089985,0.115840,-0.089799,0.482417,Normal,P72,FP-D,Normal
653,0.170927,0.281547,-0.186315,0.508857,0.264922,0.381621,-0.090961,0.442370,0.262251,-0.103593,...,0.589979,0.003480,-0.108289,0.058855,-0.129016,0.287849,Normal,P8,FP-E,Normal
654,-0.033493,-0.008705,-0.183773,0.520157,0.214841,-0.208221,-0.053012,-0.036728,0.262903,-0.113315,...,-0.131957,0.284193,-0.023389,0.234812,-0.033792,0.405667,Normal,P1,FP-A,Normal


In [12]:
os.chdir( '/home/yupaporn/codes/USAI/')
testdf = dataset
testdf.to_csv('FVvalidation_AN_RF_AN_fold8_1.csv')